# Basics of neural networks 2 (OOP)

### Making neural nets from scratch in Python using object-oriented programming

This notebook is a follow up to my NN Basics 1. This time object-oriented is used thereby easier allowing for better re-usability and net structure management. The purpose of this project was also the build neural nets without the use of NumPy, thereby gaining deeper insight into the workings of neural nets. My hope is that these will serve as my future sketches for creating NN and as such a multitude of methods will be included and possibly more will be added along the way. This means that implementing this networks in practice we should probably refrain from including all of the available methods.

In [1]:
# Relevant imports
import math
import random

### Neuron Node:

A Neuron Node including verious activation functions. 

### Currently supported activation functions:
<ul>
  <li>Step</li>
  <li>Sigmoid</li>
</ul>

In [2]:
class Neuron:
    def __init__(self, no, bias=0):
        self.no = no
        self.connections = []
        self.value = 0  # accumulated inputs (prior to activation)
        self.bias = bias
        self.activated_value = 0  # value post activation function
        self.enabled = True

    def activation(self, activation_type):
        self.value += self.bias

        if activation_type == 'sigmoid':
            self.sigmoid_activation()

        elif activation_type == 'step':
            self.step_activation()

        else:
            raise ValueError("Doesn't support this type of activation.")

    def step_activation(self):
        self.activated_value = 1 if self.value > 0 else 0

    def sigmoid_activation(self):
        self.activated_value = 1 / (1 + math.exp(-self.value))

### Synapse Node:

A Synapse Node with the possibility of being enabled/disabled.

### Currently supported methods:
<ul>
  <li>Enable synapse</li>
  <li>Disable synapse</li>
</ul>

In [3]:
class Synapse:
    def __init__(self, from_neuron, to_neuron, weight=random.uniform(-2, 2)):
        self.from_neuron = from_neuron
        self.to_neuron = to_neuron
        self.weight = weight
        self.enable = True
    
    def enable(self):
        self.enable = True
        
    def disable(self):
        self.enable = False

### Neural Network:

Our NeuralNet class allows us to initialize essentially an empty network and it's methods allows us to add different kinds of connections between those layers (like in TensorFlow - see next chapter). Furthermore it can be trained in different ways. I hope to include more possibilities going forward.

### Possible methods:
<ul>
  <li>Create input layer</li>
  <li>Create output layer</li>
  <li>Create hidden layer</li>
  <li>Create dense connetion between two layers</li>
</ul>

### Possible training:
<ul>
  <li>Backpropagation</li>
</ul>

In [4]:
class NeuralNet:
    def __init__(self):
        self.inputs = []
        self.outputs = []
        self.hidden = []  # This is a 2d list to allow for multiple hidden layers

        self.layers = []  # Neurons
        self.activations = []  # Activation function for the given layer
        self.synapses = []  # Synapses

    def add_layer(self, size, layer_type, connection='dense', activation='sigmoid', bias=0):

        # If we are creating the input layer
        if layer_type == 'input':
            self.inputs = [Neuron(i, bias) for i in range(size)]
            self.layers.append(self.inputs)
            self.activations.append(activation)

        # If we are creating the output layer
        elif layer_type == 'output':
            self.outputs = [Neuron(i, bias) for i in range(size)]

            if connection == 'dense':
                dense_connection = []

                for from_neuron in self.layers[-1]:
                    for to_neuron in self.outputs:
                        synapse = Synapse(from_neuron, to_neuron, random.uniform(-2, 2))
                        dense_connection.append(synapse)

                self.synapses.append(dense_connection)

            else:
                raise ValueError("Output layer doesn't support that connection type!")

            self.layers.append(self.outputs)
            self.activations.append(activation)

        # If we are creating a hidden layer
        elif layer_type == 'hidden':
            hidden_layer = [Neuron(i, bias) for i in range(size)]
            self.hidden.append(hidden_layer)

            if connection == 'dense':
                dense_connection = []

                for from_neuron in self.layers[-1]:
                    for to_neuron in self.hidden[-1]:
                        synapse = Synapse(from_neuron, to_neuron, random.uniform(-2, 2))
                        dense_connection.append(synapse)

                self.synapses.append(dense_connection)

            else:
                raise ValueError("This layer doesn't support that connection type!")

            self.layers.append(hidden_layer)
            self.activations.append(activation)

        else:
            raise ValueError("Network doesn't support that layer type!")

    def train(self, input_values_index, target_values_index, iterations=10000, optimizing='backpropagation'):

        for i in range(iterations):

            for index, input_values in enumerate(input_values_index):

                # Network run
                self.run(input_values)

                # backpropagation as an optimizer
                if optimizing == 'backpropagation':
                    self.backpropagation(index, target_values_index)

    def backpropagation(self, current_index, target_value_index):

        current_targets = target_value_index[current_index]

        for layer_index in range(len(self.layers) - 2, 0, -1):

            layer_error_margin = [current_targets[i] - self.layers[layer_index + 1][i].activated_value for i in
                                  range(len(self.layers[layer_index + 1]))]

            layer_derivative = [neuron.activated_value * (1 - neuron.activated_value) for neuron in
                                self.layers[layer_index + 1]]  # Derivative of the sigmoid

            current_targets = [0 for _ in range(len(self.layers[layer_index]))]
            for i, target in enumerate(current_targets):
                for synapse in self.synapses[layer_index]:
                    if synapse.from_neuron.no == i:
                        target += synapse.weight * 2 * layer_error_margin[synapse.to_neuron.no] * layer_derivative[
                            synapse.to_neuron.no]

            # This complicated step requries some explaination:
            # We are interested in finding the individual synapses' proportional influences on the cost (layer_error_margin**2) of the given layer
            # In other words we want to find the derivative of the layers cost with respect to the individual synapses leading up the that layer
            # This derivative is equal to the derivative of the cost to the activated layer (2 * layer_error_margin)
            # Times the derivative of the activated layer to the unactivated layer (val * (1 - val) --- derivative of the sigmoid, which we did above)
            # Times the derivative of the unactivated layer to synapses' influences (self.layers[layer_num] --- the previously activated layer)
            # We thus want to update our synapses with: 2 * layer_error_margin * layer_derivative * self.layers[layer_num]
            # Of course we only want to update the synapses based on its relevant connections, thus:
            for synapse in self.synapses[layer_index]:
                synapse.weight += 2 * layer_error_margin[synapse.to_neuron.no] * layer_derivative[
                    synapse.to_neuron.no] * synapse.from_neuron.activated_value

    def run(self, input_values):

        # Reset network
        self.reset()

        # Check if our training data is normalized (between 0 and 1) and if it's not we run a sigmoid on our input layer
        normalized_data = True

        for i, val in enumerate(input_values):
            if 0 >= val >= 1:
                normalized_data = False

            self.inputs[i].value = val
            self.inputs[i].activated_value = val

        if not normalized_data:
            for neuron in self.inputs:
                neuron.sigmoid_activation()

        # For every layer in the network:
        # We perform a multiplication between the neuron values in the current layer and the weights of the connections to the next layer
        # Activate the specified activation function for the next layer
        for layer_index in range(len(self.layers) - 1):

            for synapse in self.synapses[layer_index]:
                synapse.to_neuron.value += synapse.weight * synapse.from_neuron.activated_value

            for neuron in self.layers[layer_index + 1]:
                neuron.activation(self.activations[layer_index + 1])

    def reset(self):
        for layer in self.layers:
            for neuron in layer:
                neuron.value = 0
                neuron.activated_value = 0

Okay, let's instanciate a network and add some layers to our model. By default the connections are dense and they use a sigmoid activation function.

In [5]:
network = NeuralNet()
network.add_layer(3, 'input')
network.add_layer(8, 'hidden')
network.add_layer(16, 'hidden')
network.add_layer(8, 'hidden')
network.add_layer(1, 'output')

In [6]:
network.run([1, 0, 0])
print(network.outputs[0].activated_value)

0.5678350077347651


Forward feeding the network with some input and printing the output neurons activated value, we can see that our network works as intended as our synapses are randomly generated.

Let's train our network with our previous input and an expected output of 0.

In [7]:
training_data = [[1, 0, 0]]

training_labels = [[0]]

network.train(training_data, training_labels, iterations=10000)

In [8]:
network.run([1, 0, 0])
print(network.outputs[0].activated_value)

0.004945296747181074


We see now how the synapses has gravitated the network toward an output of 0 and when we again feed our input, we get a pretty acturate output.

We can futher train the network train the network in the other direction if we so disire.

In [10]:
training_data = [[1, 0, 0]]

training_labels = [[1]]

network.train(training_data, training_labels, iterations=10000)

In [11]:
network.run([1, 0, 0])
print(network.outputs[0].activated_value)

0.9950111194658214


This might initially look very promising, however the way we have trained this network only makes is gravitate towards a certain output i.e. regardless of input the output will always be near 0.

Weather this is due to some error or misunderstanding on my part regarding backpropagation or it occurs as a result of not yet having implemented a stochastic gradient decent I am not sure. But investigation will follow.